# SQFD

## Reference

This refers to the following paper by Beeks, Uysal and Seidl:
> C. Beecks, M. S. Uysal, and T. Seidl. 
> Signature quadratic form distance. 
> In Proc. ACM CIVR, pages 438–445, 2010.

## Problem statement

The idea here is to define a distance between weighted feature sets $\mathcal{X} = \{ (x_i, w_{x_i}) \}$ and $\mathcal{Y} = \{ (y_i, w_{y_i}) \}$:

$$d_\text{SQFD}(\mathcal{X}, \mathcal{Y})^2 = (w_\mathcal{X} \, | \, -w_\mathcal{Y}) \cdot M \cdot (w_\mathcal{X} \, | \, -w_\mathcal{Y})^T.$$

where $w_\mathcal{X}$ and $w_\mathcal{Y}$ are weight vectors and $M$ is the following block matrix:

$$M = \left( \begin{array}{ccc}
   M_{\mathcal{X}, \mathcal{X}} & M_{\mathcal{X}, \mathcal{Y}} \\
   {M_{\mathcal{Y}, \mathcal{X}}} & M_{\mathcal{Y}, \mathcal{Y}} 
\end{array} \right)$$

where:

$$\forall (i, j), \, (M_{\mathcal{X}, \mathcal{X}})_{i,j} = k(x_i,x_j)$$
$$\forall (i, j), \, (M_{\mathcal{X}, \mathcal{Y}})_{i,j} = (M_{\mathcal{Y}, \mathcal{X}})_{j, i} = k(x_i,y_j)$$
$$\forall (i, j), \, (M_{\mathcal{Y}, \mathcal{Y}})_{i,j} = k(y_i,y_j),$$

where $k$ is a local kernel.

## My use of this distance

I start with coding this (squared) distance in Python, with $k$ being a RBF kernel:

In [1]:
import numpy
from sklearn.metrics.pairwise import rbf_kernel

def sqfd_sq(X, Y, alpha, w_X=None, w_Y=None):
    if w_X is None:
        w_X = 1. / X.shape[0] * numpy.ones((X.shape[0],))
    if w_Y is None:
        w_Y = 1. / Y.shape[0] * numpy.ones((Y.shape[0],))
    XY = numpy.concatenate((X, Y))
    m = rbf_kernel(XY, XY, gamma=alpha)
    w = numpy.concatenate((w_X, -w_Y))
    return numpy.dot(numpy.dot(w, m), w)

Note that, by default, weights are set to $1/n$ for a set of size $n$.

In the following, I will focus on this equal weight case.

## Developping the distance calculation

In the following, I will also assume that $|\mathcal{X}| = n$ and $|\mathcal{Y}| = m$.

Then, we get:
$$d_\text{SQFD}(\mathcal{X}, \mathcal{Y})^2 = \frac{1}{n^2}\sum_{i=1}^n\sum_{j=1}^n k(x_i, x_j) + \frac{1}{m^2}\sum_{i=1}^m\sum_{j=1}^m k(y_i, y_j) - \frac{2}{m \cdot n}\sum_{i=1}^n\sum_{j=1}^m k(x_i, y_j)$$

If we denote by $K$ the match kernel, we can re-write this as:

$$d_\text{SQFD}(\mathcal{X}, \mathcal{Y})^2 = K(\mathcal{X}, \mathcal{X}) + K(\mathcal{Y}, \mathcal{Y}) - 2 K(\mathcal{X}, \mathcal{Y}).$$

In other words, $d_\text{SQFD}$ is the distance between feature sets embedded in the RKHS associated to $K$.